In [ ]:
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime

# ディレクトリのパス
directory_path = '/content/drive/My Drive/samples-20231212T112538Z-001/samples/order_new'

# 指定したディレクトリ内のExcelファイルのパスをリストとして取得
excel_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.xlsx')]
print(excel_files)
# 各店からの注文をまとめるデータフレームを初期化
orders_combined = pd.DataFrame()

# Excelファイルを読み込んで注文をまとめる
for file_path in excel_files:
    df = pd.read_excel(file_path)  # Excelファイルを読み込む
    orders_combined = pd.concat([orders_combined, df], axis=0)  # 注文をまとめる

# 各商品の合計注文量を計算する
total_orders = orders_combined.sum()

# 合計注文量を表示する
print("各商品の合計注文量:")
print(total_orders)

import pandas as pd

# 在庫表のファイルパス
file_path = '/content/drive/My Drive/samples-20231212T112538Z-001/samples/inventory.xlsx'

# Excelファイルを読み込む
df_inventory = pd.read_excel(file_path)

# 最終行の情報を抽出して表示する
latest_inventory = df_inventory.tail(1)  # 最終行のデータを取得
latest_inventory = latest_inventory.drop(columns=['日付', '曜日'])
print("最新の野菜の在庫情報:")
for col in latest_inventory.columns:
    print(f"{col}: {latest_inventory[col].values[0]}")

# 新しいDataFrameを作成して差分を計算し、differenceデータフレームに格納
difference = pd.DataFrame(columns=['Vegetable', 'Difference'])

# 最新の野菜の在庫情報の要素と値を出力
for col in latest_inventory.columns:
    diff = latest_inventory[col].values[0] - total_orders[col].sum()
    difference = difference.append({'Vegetable': col, 'Difference': diff}, ignore_index=True)

# 各野菜の合計注文数と最新の在庫情報の差分を計算
difference = latest_inventory.values[0] - total_orders.values

# 差分を表示
print("各野菜の合計注文数と在庫情報の差分:")
for i, col in enumerate(total_orders.index):
    print(f"{col}: {difference[i]}")

from google.colab import drive
import pandas as pd

# Google Driveをマウント
drive.mount('/content/drive')

# pickup.xlsx のパス
pickup_file_path = '/content/drive/My Drive/samples-20231212T112538Z-001/samples/pickup.xlsx'

# "pickup.xlsx"からしきい値と追加量を読み込む
data = pd.read_excel(pickup_file_path, index_col=0)  # 一列目をインデックスとして読み込む

# しきい値を取得し出力
thresholds = data.loc['しきい値']
print("しきい値:")
for vegetable, threshold in thresholds.items():
    print(f"{vegetable}: {threshold}")

# 追加量を取得し出力
additional_amounts = data.loc['追加量']
print("\n追加量:")
for vegetable, amount in additional_amounts.items():
    print(f"{vegetable}: {amount}")

# 差分を計算
difference = latest_inventory.values[0] - total_orders.values
print(difference)

# 各野菜のしきい値を取得
thresholds = data.loc['しきい値']

# 発注が必要な野菜とその発注数を特定
orders_needed = {}
for i, col in enumerate(total_orders.index):
    if difference[i] < thresholds[col]:
        orders_needed[col] = thresholds[col] - difference[i]

# 発注が必要な野菜と発注数を表示
print("\n発注が必要な野菜とその発注数:")
for veg, order_qty in orders_needed.items():
    print(f"{veg}: {order_qty}")

# メール送信
body = '以下の野菜を発注します。よろしくお願いします。\n\n'
for veggie, quantity in orders_needed.items():
    body += f'{veg}: {order_qty}個\n'

msg = MIMEMultipart()
msg['From'] = 'zhenmeizijianye@gmail.com'
msg['To'] = 'zhenmeizijianye@gmail.com'
msg['Subject'] = '野菜の発注について'
msg.attach(MIMEText(body, 'plain'))

try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login('zhenmeizijianye@gmail.com', 'oldm qiyi svru kohk')
    server.sendmail('zhenmeizijianye@gmail.com', 'zhenmeizijianye@gmail.com', msg.as_string())
    print('メールが送信されました！')
except Exception as e:
    print(f'エラーが発生しました: {str(e)}')
finally:
    server.quit()

from datetime import datetime

# Google Driveをマウント
drive.mount('/content/drive')
# pickup.xlsx のパスと最新の野菜の在庫情報のパス
pickup_file_path = '/content/drive/My Drive/samples-20231212T112538Z-001/samples/pickup.xlsx'
inventory_file_path = '/content/drive/My Drive/samples-20231212T112538Z-001/samples/inventory.xlsx'

# 最新の野菜の在庫情報を読み込む
inventory = pd.read_excel(inventory_file_path)

# しきい値と追加量を取得
data = pd.read_excel(pickup_file_path, index_col=0)  # 一列目をインデックスとして読み込む
thresholds = data.loc['しきい値']
additional_amounts = data.loc['追加量']

# 最新の野菜の在庫情報の差分を計算
latest_inventory = inventory.iloc[-1]  # 最新の在庫情報
difference = {}
for col in latest_inventory.index:
    if col in thresholds.index:  # 商品ごとの注文合計数のDataFrameに存在する場合
        diff = latest_inventory[col] - total_orders[col].sum()
        difference[col] = diff

# 新しい行を作成して、最新の在庫情報を更新
date_today = datetime.now().date()
latest_inventory_date = {'日付': date_today}
latest_inventory_date['曜日'] = date_today.strftime('%a')  # 曜日を取得して追加する

for col, diff in difference.items():
    latest_inventory_date[col] = max(0, latest_inventory[col] - diff)

# DataFrameに新しい行を追加
inventory = inventory.append(latest_inventory_date, ignore_index=True)

# 更新された在庫情報をExcelファイルに上書き保存
inventory.to_excel(inventory_file_path, index=False)
print("最新の在庫情報が上書き保存されました。")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['/content/drive/My Drive/samples-20231212T112538Z-001/samples/order_new/order_B_20230524.xlsx', '/content/drive/My Drive/samples-20231212T112538Z-001/samples/order_new/order_C_20230524.xlsx', '/content/drive/My Drive/samples-20231212T112538Z-001/samples/order_new/order_D_20230524.xlsx', '/content/drive/My Drive/samples-20231212T112538Z-001/samples/order_new/order_A_20230524.xlsx']
各商品の合計注文量:
レタス      42.0
白菜       25.0
ほうれん草    23.0
大根       15.0
ニンジン     32.0
トマト      31.0
キャベツ     21.0
dtype: float64
最新の野菜の在庫情報:
トマト: 31
キャベツ: 21
レタス: 42
白菜: 25
ほうれん草: 23
大根: 15
ニンジン: 32
各野菜の合計注文数と在庫情報の差分:
レタス: -11.0
白菜: -4.0
ほうれん草: 19.0
大根: 10.0
ニンジン: -9.0
トマト: -16.0
キャベツ: 11.0


<ipython-input-9-f20ced97a600>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  difference = difference.append({'Vegetable': col, 'Difference': diff}, ignore_index=True)
<ipython-input-9-f20ced97a600>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  difference = difference.append({'Vegetable': col, 'Difference': diff}, ignore_index=True)
<ipython-input-9-f20ced97a600>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  difference = difference.append({'Vegetable': col, 'Difference': diff}, ignore_index=True)
<ipython-input-9-f20ced97a600>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  difference = difference.append({'Vegetable': col, 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
しきい値:
トマト: 50
キャベツ: 40
レタス: 50
白菜: 30
ほうれん草: 40
大根: 30
ニンジン: 40

追加量:
トマト: 100
キャベツ: 80
レタス: 100
白菜: 60
ほうれん草: 80
大根: 60
ニンジン: 80
[-11.  -4.  19.  10.  -9. -16.  11.]

発注が必要な野菜とその発注数:
レタス: 61.0
白菜: 34.0
ほうれん草: 21.0
大根: 20.0
ニンジン: 49.0
トマト: 66.0
キャベツ: 29.0
メールが送信されました！
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
最新の在庫情報が上書き保存されました。


<ipython-input-9-f20ced97a600>:159: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inventory = inventory.append(latest_inventory_date, ignore_index=True)
